In [2]:
import pickle
import joblib
import pandas as pd
import numpy as np

In [3]:
insurance_predictor= pickle.load(open ("knn_model.pickle",'rb'))

In [4]:
insurance_predictor

RandomizedSearchCV(cv=5, estimator=KNeighborsRegressor(),
                   param_distributions={'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10,
                                                        11, 12, 13, 14, 15, 16,
                                                        17, 18, 19, 20, 21, 22,
                                                        23, 24, 25, 26, 27, 28,
                                                        29],
                                        'p': [1, 2]})

In [4]:
test_ds=pd.read_csv("for_test_insu.csv")

In [6]:
test_ds

,Unnamed: 0,age,sex,bmi,children,smoker,charges,r_northeast,r_northwest,r_southeast,r_southwest
0,888,22,0,39.500,0,0,1682.59700,0,0,0,1
1,889,57,0,33.630,1,0,11945.13270,0,1,0,0
2,890,64,1,26.885,0,1,29330.98315,0,1,0,0
3,891,36,1,29.040,4,0,7243.81360,0,0,1,0
4,892,54,0,24.035,0,0,10422.91665,1,0,0,0
5,893,47,0,38.940,2,1,44202.65360,0,0,1,0


In [7]:
test_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   6 non-null      int64  
 1   age          6 non-null      int64  
 2   sex          6 non-null      int64  
 3   bmi          6 non-null      float64
 4   children     6 non-null      int64  
 5   smoker       6 non-null      int64  
 6   charges      6 non-null      float64
 7   r_northeast  6 non-null      int64  
 8   r_northwest  6 non-null      int64  
 9   r_southeast  6 non-null      int64  
 10  r_southwest  6 non-null      int64  
dtypes: float64(2), int64(9)
memory usage: 656.0 bytes


In [8]:
x=test_ds.drop(['charges'],axis=1)
y=test_ds['charges']

In [9]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   6 non-null      int64  
 1   age          6 non-null      int64  
 2   sex          6 non-null      int64  
 3   bmi          6 non-null      float64
 4   children     6 non-null      int64  
 5   smoker       6 non-null      int64  
 6   r_northeast  6 non-null      int64  
 7   r_northwest  6 non-null      int64  
 8   r_southeast  6 non-null      int64  
 9   r_southwest  6 non-null      int64  
dtypes: float64(1), int64(9)
memory usage: 608.0 bytes


In [10]:
x.drop(['Unnamed: 0'],axis=1,inplace=True)

In [11]:
y_pred=insurance_predictor.predict(x)

In [12]:
y_pred

array([13007.56004 , 13656.618972, 32148.657084, 16890.553268,
       13961.883684, 45731.01816 ])

In [13]:
from sklearn.metrics import roc_auc_score,r2_score,mean_squared_error

In [14]:
r2_score(y,y_pred)

0.8083928018437616

In [15]:
x.head(2)

,age,sex,bmi,children,smoker,r_northeast,r_northwest,r_southeast,r_southwest
0,22,0,39.50,0,0,0,0,0,1
1,57,0,33.63,1,0,0,1,0,0


In [29]:
#predict charges (age= 22,sex= male, bmi= 39.45,childern= 2,smoker=no, area= northwest)
def predict_charges(age,gender,bmi,children,smoker, region):
    location_index= x.columns.get_loc(region)
    z=np.zeros(len(x.columns))
    z[0]= age
    
    if gender == 'male': 
        z[1] = 1
    else:
        z[1] = 0
        
    z[2] = bmi
    z[3] = children
    
    if smoker == 'yes': 
        z[4] = 1
    else:
        z[4] = 0 
    z[location_index]= 1
    
    return insurance_predictor.predict([z])[0]
    

In [30]:
predict_charges(29,'male',39,0,'yes','r_southwest')

35565.432100000005

In [31]:
predict_charges(34,'female',39,1,'no','r_northeast')

13623.149872999998

In [32]:
import json
columns= {
    'data_coumns': [col.lower() for col in x.columns]
}
with open("columns1.json","w") as f:
    f.write(json.dumps(columns))